# Segmenting and Clustering Neighborhoods in Toronto

## Introduction

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#menu1">Download and Explore Dataset</a>

2. <a href="#menu2">Explore Neighborhoods in New York City</a>

3. <a href="#menu3">Analyze Each Neighborhood</a>

4. <a href="#menu4">Cluster Neighborhoods</a>

5. <a href="#menu5">Examine Clusters</a>    
</font>
</div>

Importing libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
import geopandas as gpd #geospatial data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='menu1'></a>

## 1. Download and Explore Dataset

In [19]:
gpd_geojson = gpd.read_file('data/Police Boundaries Data.geojson')
gpd_geojson.head()

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,115,25832759,25832759,25832760,4,14,14,14,None,None,-79.417189,43.649990,15976289,2.710943e+07,30015.196591,(POLYGON ((-79.40506136065933 43.6330695853511...
1,116,25832817,25832817,25832797,3,43,43,43,None,None,-79.199932,43.766467,15976321,1.137953e+08,53902.463203,(POLYGON ((-79.1341405763917 43.77397487161847...
2,117,25832819,25832819,25832820,1,31,31,31,None,None,-79.519215,43.748497,15976353,8.177649e+07,44427.808912,(POLYGON ((-79.58064388180388 43.7630179182295...
3,118,25832822,25832822,25832820,2,32,32,32,None,None,-79.432549,43.756900,15976385,1.170648e+08,43667.613471,(POLYGON ((-79.38145443613818 43.7302410973748...
4,119,25832825,25832825,25832820,3,33,33,33,None,None,-79.352777,43.764411,15976417,1.012368e+08,45137.976238,(POLYGON ((-79.31979414655423 43.7683633049311...


In [5]:
gpd_toronto = gpd.read_file('data/WARD_WGS84.shp')
gpd_toronto.head()

,AREA_ID,AREA_TYPE,AREA_S_CD,AREA_L_CD,AREA_NAME,X,Y,LONGITUDE,LATITUDE,geometry
0,2551040,WD18,16,16,Don Valley East,318237.290,4844000.0,-79.33298,43.739716,"POLYGON ((-79.31334536599999 43.716986164, -79..."
1,2551044,WD18,03,03,Etobicoke-Lakeshore,303099.474,4831000.0,-79.52087,43.621646,"POLYGON ((-79.497766447 43.651983193, -79.4972..."
2,2551048,WD18,15,15,Don Valley West,314825.876,4843000.0,-79.37536,43.728396,"POLYGON ((-79.352321171 43.715734879, -79.3520..."
3,2551052,WD18,23,23,Scarborough North,324522.149,4852000.0,-79.25467,43.809672,"POLYGON ((-79.22590696100001 43.839601785, -79..."
4,2551056,WD18,11,11,University-Rosedale,313306.543,4837000.0,-79.39432,43.671139,"POLYGON ((-79.390039544 43.690503926, -79.3900..."


In [18]:
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

geojson_toronto = gpd_toronto.to_crs(epsg=4326)
toronto = folium.features.GeoJson(geojson_toronto)

map_toronto.add_child(toronto)

map_toronto


<a id='menu2'></a>

## 2. Explore Neighborhoods in City of Toronto

<a id='menu3'></a>

## 3. Analyze Each Neighborhood

<a id='menu4'></a>

## 4. Cluster Neighborhoods

<a id='menu5'></a>

## 5. Examine Clusters